In [2]:
import numpy as np
import pandas as pd

from fastparquet import ParquetFile,write
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from cm2df import cm2df,precision_recall_fscore_support_metrics2df
from sklearn.metrics import confusion_matrix, classification_report,precision_recall_fscore_support
import pickle
from bson.json_util import dumps
from tqdm import tqdm
import re

In [3]:
filename = 'finalized_model111819.sav'
# loading the model from disk
model = pickle.load(open(filename, 'rb'))

In [4]:
usecolsX=['category', 'past', 'is_eventbrite', 'is_free', 'doors',
       'sold_out', 'venue.id', 'venue.popularity', 'venue.zip',
       'ticket_allages', 'ticket_price_low', 'ticket_price_max', 'min_age',
       'artist.popularity.sum', 'artist.popularity.avg',
       'artist.popularity.max', 'dow', 'doy', 'month', 'day', 'hour',
       'venue.tol_num_events', 'duration', 'duration_day', 'multiday']
usecoly1=['votes']
usecoly2=['avg_votes_pday']

### read from daily do512 json file since collected database does not contain some info for features

In [5]:
# start off the dataframe 
ncolumns=['id','event_name','category','past', 'is_eventbrite', 'is_free', 'doors','sold_out', 'venue.id', 'venue.popularity', 'venue.zip','ticket_info', 'artist','tz_adjusted_begin_date','tz_adjusted_end_date','votes']
ev_df=pd.DataFrame(columns=ncolumns)

In [6]:
# %load NM_get_do512_future_data.py
# Get the event data from the do 512 website for the future N days
import time
from datetime import datetime, timedelta
import json
import requests
from pandas.io.json import json_normalize

def get_request_url(days_to_add,page):
    base_url = 'https://do512.com/events/'
    date = datetime.now() + timedelta(days=days_to_add)
    print(date)
    url_suffix = str(date.year) + '/' + str(date.month) + '/' + str(date.day) + '.json'
    page_num = '?page=' + str(page)
    request_url = base_url + url_suffix + page_num
    return request_url


def get_events(days_to_add,page):
    request_url = get_request_url(days_to_add,page)
    response = requests.get(request_url)
    return response

def process_response(response):
    events = json.loads(response)["events"]
    return events

def get_total_pages(response):
    total_pages = json.loads(response)["paging"]["total_pages"]
    return total_pages

def date_to_str(date):
    return str(date.month) + '-' + str(date.day) + '-' + str(date.year)

def get_file_name(NumDaysInFut):
    folder_name = 'C:\SiemensCodeSpace\Eventful\PredictionModels\data\\'
    base_name = 'do512_'
    name_suffix = date_to_str(datetime.now()) + "-"  + date_to_str(datetime.now() + timedelta(days=(NumDaysInFut-1)))
    file_name = folder_name + base_name + name_suffix + '.parquet'
    return file_name

In [8]:
query= get_events(-19,1)

2019-11-01 15:39:27.240850


In [9]:
pages= query.json()['paging']['total_pages']

In [10]:
for p in range(pages):
    query= get_events(-19,p+1)
    content=query.json()
    events=content['events']
    print("number of events on the page:",len(events))
    for evt in events:
        # new list
        ev_list=[]
        ev_list+=[evt['id']]
        ev_list+=[evt['title']]
        ev_list+=[evt['category']]
        ev_list+=[evt['past']]
        ev_list+=[evt['is_eventbrite']]
        ev_list+=[evt['is_free']]
        ev_list+=[evt['doors']]
        ev_list+=[evt['sold_out']]
        ev_list+=[evt['venue']['id']]
        ev_list+=[evt['venue']['popularity']]
        ev_list+=[evt['venue']['zip']]
        ev_list+=[evt['ticket_info']]
        ev_list+=[evt['artists']]
        ev_list+=[evt['tz_adjusted_begin_date']]
        ev_list+=[evt['tz_adjusted_end_date']]
        ev_list+=[evt['votes']]
        #save list to dataframe
        ev_df.loc[len(ev_df.index)] = ev_list    

2019-11-01 15:39:51.201924
number of events on the page: 25
2019-11-01 15:39:51.732251
number of events on the page: 25
2019-11-01 15:39:52.695166
number of events on the page: 25
2019-11-01 15:39:53.487780
number of events on the page: 25


In [11]:
print("total number of events:",len(ev_df))

total number of events: 100


In [12]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,venue.zip,ticket_info,artist,tz_adjusted_begin_date,tz_adjusted_end_date,votes
0,10093864,Formula 1 United States Grand Prix ft. P!NK & ...,Sports + Activities,True,False,False,False,False,25690,10.0,78617,,"[{'id': 153112, 'title': 'P!nk', 'permalink': ...",2019-11-01T13:55:00-05:00,2019-11-04T02:00:00-06:00,5307
1,10410048,"The Chainsmokers w/ 5 Seconds of Summer, Lenno...",Music,True,False,False,True,False,596,1.0,78701,$29.50-$99.50,"[{'id': 441971, 'title': 'THE CHAINSMOKERS', '...",2019-11-01T18:00:00-05:00,2019-11-02T02:00:00-05:00,487
2,10871100,Noah Gundersen,Music,True,True,False,False,False,119,7.0,78701,21+ / $24,"[{'id': 207598, 'title': 'Noah Gundersen', 'pe...",2019-11-01T19:00:00-05:00,2019-11-02T02:00:00-05:00,210
3,10874324,"GWAR w/ Sacred Reich, Toxic Holocaust & More",Music,True,False,False,True,False,251731,15.0,78701,All Ages | $5 minor surcharge,"[{'id': 39773, 'title': 'Gwar', 'permalink': '...",2019-11-01T19:00:00-05:00,2019-11-02T02:00:00-05:00,199
4,10679378,Jade Bird w/ Flyte,Music,True,False,False,True,False,654,15.0,78702,All Ages,"[{'id': 1827972, 'title': 'Jade Bird', 'permal...",2019-11-01T19:30:00-05:00,2019-11-02T02:00:00-05:00,156


In [13]:
vzip=ev_df['venue.zip'].values
# cleanup the zipcodes
vzip=[str(x).split(' ')[0] for x in vzip]
vzip=['' if x=='Austin' else x for x in vzip]
vzip=['' if x=='nan' else x for x in vzip]
vzip=['' if len(x)<5 else x for x in vzip]

In [14]:
vzip=[int(x) if x.isdigit() else 0 for x in vzip]

In [15]:
ev_df['venue.zip']=vzip

In [16]:
# clean up ticket infomation
# insert new columns
ev_df['ticket_allages']=[False for x in range(len(ev_df))]
ev_df['ticket_price_low']=[0 for x in range(len(ev_df))]
ev_df['ticket_price_max']=[0 for x in range(len(ev_df))]
ev_df['min_age']=[None for x in range(len(ev_df))]

# parse ticket info string
tkt_str=ev_df.ticket_info.values

# dump all info of ticket into other columns
for i,item in tqdm(enumerate(tkt_str)):
    if i%1000==0:print(i)
    #check 'free'
    if "free" in item.lower() : ev_df.loc[i,'is_free']=True
    # check child ok        
    if ("all ages" in item.lower()) or ("kids" in item.lower()) or ("child" in item.lower()): ev_df.loc[i,'ticket_allages']=True
    
    #replace all whitespace after '$'    
    item = re.sub('\$ ','$',item)
    prices=re.findall('(?<=\$)\d*\.?\d*', item)
    # get all lower/upper $number
    if prices: # not [] or None
        if prices[0].isdigit():
            if len(prices)==1: 
                ev_df.loc[i,'ticket_price_low']=prices[0]
                ev_df.loc[i,'ticket_price_max']=None
            else:
                #convert all prices to numbers , in case any of them not number
                nprices=[float(x) for x in prices if x.isdigit()]
                
                ev_df.loc[i,'ticket_price_low']=min(nprices)
                ev_df.loc[i,'ticket_price_max']=max(nprices)      
    # get minimum age
    min_ag=re.findall(r"(\d+)\+",item)
    if min_ag:
        ev_df.loc[i,'min_age']=int(min_ag[0])
    else:ev_df.loc[i,'min_age']=None

0it [00:00, ?it/s]

0


100it [00:00, 1517.84it/s]


In [17]:
# parse artist information
# get artist column andef
e_artist=ev_df.artist.values
#get artist_popularity
p_art_avg=[]
p_art_sum=[]
p_art_max=[]
for item in e_artist:
    if len(item)==0:
        p_art_sum+=[0]
        p_art_avg+=[0]
        p_art_max+=[0]
    elif len(item)==1:
        p_art_sum+=[json.loads(dumps(item))[0]['popularity']]
        p_art_avg+=[json.loads(dumps(item))[0]['popularity']]
        p_art_max+=[json.loads(dumps(item))[0]['popularity']]
    else:
        subsum=0
        partmax=0
        for sub in item:
            part=json.loads(dumps(item))[0]['popularity']
            subsum+=part
            if part> partmax: partmax=part
        p_art_sum+=[subsum]
        p_art_avg+=[subsum/len(item)]
        p_art_max+=[partmax]

In [20]:
ev_df['artist.popularity.sum']=p_art_sum
ev_df['artist.popularity.avg']=p_art_avg
ev_df['artist.popularity.max']=p_art_max

In [29]:
# setting the time and get the time features
ev_df['tz_adjusted_begin_date']=pd.to_datetime(ev_df['tz_adjusted_begin_date'],utc = True)
ev_df['tz_adjusted_end_date']=pd.to_datetime(ev_df['tz_adjusted_end_date'],utc = True)


ev_df['duration']=(ev_df['tz_adjusted_end_date']-ev_df['tz_adjusted_begin_date']).apply(lambda x:x/ np.timedelta64(1, 'h'))
ev_df['duration_day']=ev_df['duration'].apply(lambda x:np.ceil(x/24))
a=ev_df['duration'].values/24
multid=[True if x >=1 else False for x in a]
ev_df['multiday']=multid
# fix duration_day=0
ev_df=ev_df.replace({'duration':0},2)
ev_df=ev_df.replace({'duration_day':0},1)

In [30]:
ev_df['dow']=ev_df['tz_adjusted_begin_date'].dt.dayofweek
ev_df['doy']=ev_df['tz_adjusted_begin_date'].dt.dayofyear
ev_df['month']=ev_df['tz_adjusted_begin_date'].dt.month
ev_df['day']=ev_df['tz_adjusted_begin_date'].dt.day
ev_df['hour']=ev_df['tz_adjusted_begin_date'].dt.hour

In [31]:
### drop some redundant columns
ev_df.drop(['ticket_info', 'artist','tz_adjusted_begin_date','tz_adjusted_end_date'], axis=1, inplace=True)

In [32]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,...,artist.popularity.avg,artist.popularity.max,duration,duration_day,multiday,dow,doy,month,day,hour
0,10093864,Formula 1 United States Grand Prix ft. P!NK & ...,Sports + Activities,True,False,False,False,False,25690,10.0,...,47.0,47.0,61.083333,3.0,True,4,305,11,1,18
1,10410048,"The Chainsmokers w/ 5 Seconds of Summer, Lenno...",Music,True,False,False,True,False,596,1.0,...,2070.0,2070.0,8.000000,1.0,False,4,305,11,1,23
2,10871100,Noah Gundersen,Music,True,True,False,False,False,119,7.0,...,1145.0,1145.0,7.000000,1.0,False,5,306,11,2,0
3,10874324,"GWAR w/ Sacred Reich, Toxic Holocaust & More",Music,True,False,False,True,False,251731,15.0,...,10471.0,10471.0,7.000000,1.0,False,5,306,11,2,0
4,10679378,Jade Bird w/ Flyte,Music,True,False,False,True,False,654,15.0,...,119.0,119.0,6.500000,1.0,False,5,306,11,2,0


In [33]:
### Add the total events host at the venue , query from past year library
# load back training file
toevv=pd.read_csv('Dataset_Lib/TotalEventsPerVenueID2018.csv')

In [34]:
toevv=toevv.set_index('id')

In [35]:
# query venue id then input number
tolnev=[]
venid=ev_df['venue.id'].values
lvenid=toevv['venue.id'].values
for idn in venid:
    if idn in lvenid:tolnev+=[toevv.loc[toevv['venue.id']==idn].iloc[0,0]]
    else:tolnev+=[1]

In [36]:
ev_df['venue.tol_num_events']=tolnev

In [37]:
ev_df.past=ev_df.past.astype(bool)
ev_df.is_eventbrite=ev_df.is_eventbrite.astype(bool)
ev_df.doors=ev_df.doors.astype(bool)
ev_df.sold_out=ev_df.sold_out.astype(bool)

In [38]:
ev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 28 columns):
id                       100 non-null object
event_name               100 non-null object
category                 100 non-null object
past                     100 non-null bool
is_eventbrite            100 non-null bool
is_free                  100 non-null bool
doors                    100 non-null bool
sold_out                 100 non-null bool
venue.id                 100 non-null object
venue.popularity         100 non-null float64
venue.zip                100 non-null int64
votes                    100 non-null object
ticket_allages           100 non-null bool
ticket_price_low         100 non-null object
ticket_price_max         81 non-null float64
min_age                  14 non-null object
artist.popularity.sum    100 non-null float64
artist.popularity.avg    100 non-null float64
artist.popularity.max    100 non-null float64
duration                 100 non-null float64
durat

In [39]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,...,artist.popularity.max,duration,duration_day,multiday,dow,doy,month,day,hour,venue.tol_num_events
0,10093864,Formula 1 United States Grand Prix ft. P!NK & ...,Sports + Activities,True,False,False,False,False,25690,10.0,...,47.0,61.083333,3.0,True,4,305,11,1,18,27
1,10410048,"The Chainsmokers w/ 5 Seconds of Summer, Lenno...",Music,True,False,False,True,False,596,1.0,...,2070.0,8.000000,1.0,False,4,305,11,1,23,50
2,10871100,Noah Gundersen,Music,True,True,False,False,False,119,7.0,...,1145.0,7.000000,1.0,False,5,306,11,2,0,460
3,10874324,"GWAR w/ Sacred Reich, Toxic Holocaust & More",Music,True,False,False,True,False,251731,15.0,...,10471.0,7.000000,1.0,False,5,306,11,2,0,405
4,10679378,Jade Bird w/ Flyte,Music,True,False,False,True,False,654,15.0,...,119.0,6.500000,1.0,False,5,306,11,2,0,237


In [40]:
# rearrange column orders:
ev_df=ev_df[usecolsX+['votes']]

In [41]:
ev_df.to_csv('Dataset_Lib/ev_do512_102919.csv')

#### Null Data cleanup

In [44]:
ev_df.isnull().sum()

category                 0
past                     0
is_eventbrite            0
is_free                  0
doors                    0
sold_out                 0
venue.id                 0
venue.popularity         0
venue.zip                0
ticket_allages           0
ticket_price_low         0
ticket_price_max         0
min_age                  0
artist.popularity.sum    0
artist.popularity.avg    0
artist.popularity.max    0
dow                      0
doy                      0
month                    0
day                      0
hour                     0
venue.tol_num_events     0
duration                 0
duration_day             0
multiday                 0
votes                    0
dtype: int64

In [43]:
ev_df.min_age=ev_df.min_age.fillna(0)
ev_df.ticket_price_max=ev_df.ticket_price_max.fillna(0)
ev_df.ticket_price_low=ev_df.ticket_price_low.fillna(0)

#### Ecoding data for input:
(1) Convert boolean columns
(2) Convert/map category,
(3) Map the venue id to nullify
(4) Map the zipcode to nullify

In [45]:
boolcol = ev_df.columns[(ev_df.dtypes.values == np.dtype('bool'))]
## convert all boolean columns into 0/1
for cols in boolcol:
    ev_df[cols]=ev_df[cols].astype(int)

In [46]:
### Conversion of the features
# mapping of the category
mapcat={'Music ': 0, 'Comedy': 1, 'Happy Hour + Drink Specials': 2, "DJ's + Parties": 3, 'Film + TV': 4, 'Karaoke + Trivia': 5, 'Workshops + Classes': 6, 'Community': 7, 'Literary': 8, 'Art & Culture': 9, 'Sports + Activities': 10, 'Food + Drink': 11, 'LGBTQ+': 12, 'Theater': 13, 'Variety / Other': 14, 'Activism': 15, 'Exhibit': 16, 'Opening': 17, 'Fashion': 18, 'Free Week': 19}

for i,line in enumerate(mapcat):
    ev_df.category.replace(line,i,inplace=True)

In [47]:
# load back the venue id library
encode_dict=np.load('Dataset_Lib/venue_id_lib.npy',allow_pickle=True).item()

In [48]:
ev_df['venue.id']=ev_df['venue.id'].map(encode_dict)

In [49]:
nct=max(encode_dict.values())
fullvn_id=ev_df['venue.id'].values
nvn_id=[]
for item in fullvn_id:
    if np.isnan(item):
        nvn_id+=[int(nct+1)]
        nct+=1
    else: nvn_id+=[int(item)]

In [50]:
ev_df['venue.id']=nvn_id  # place back the venue id nullified

In [51]:
## load back the zip code library # neutrolize the venue zips:
encode_zip_dict=np.load('Dataset_Lib/venue_zip_lib.npy',allow_pickle=True).item()
ev_df['venue.zip']=ev_df['venue.zip'].map(encode_zip_dict)

In [52]:
# clean up the nan
nct=max(encode_zip_dict.values())
fullvn_zip=ev_df['venue.zip'].values
nvn_zip=[]
for item in fullvn_zip:
    if np.isnan(item):
        nvn_zip+=[int(nct+1)]
        nct+=1
    else: nvn_zip+=[int(item)]

In [53]:
ev_df['venue.zip']=nvn_zip

In [54]:
ev_df

,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,venue.zip,ticket_allages,...,dow,doy,month,day,hour,venue.tol_num_events,duration,duration_day,multiday,votes
0,10,1,0,0,0,0,270,10.0,48,0,...,4,305,11,1,18,27,61.083333,3.0,1,5307
1,0,1,0,0,1,0,139,1.0,0,0,...,4,305,11,1,23,50,8.000000,1.0,0,487
2,0,1,1,0,0,0,6,7.0,0,0,...,5,306,11,2,0,460,7.000000,1.0,0,210
3,0,1,0,0,1,0,11,15.0,0,1,...,5,306,11,2,0,405,7.000000,1.0,0,199
4,0,1,0,0,1,0,29,15.0,2,1,...,5,306,11,2,0,237,6.500000,1.0,0,156
5,0,1,0,0,1,1,9,15.0,0,0,...,4,305,11,1,22,404,9.000000,1.0,0,96
6,1,1,1,0,1,0,28,1.0,0,0,...,5,306,11,2,0,244,1.750000,1.0,0,120
7,0,1,0,0,1,0,15,15.0,0,1,...,4,305,11,1,23,377,8.000000,1.0,0,117
8,0,1,0,0,1,1,9,15.0,0,0,...,5,306,11,2,1,404,6.000000,1.0,0,95
9,1,1,0,1,0,0,1614,1.0,3,0,...,5,306,11,2,1,1,5.500000,1.0,0,128


In [55]:
# Load back to the X scalar
import joblib
scalar = joblib.load('MinMaxScalar_111819.pkl')

In [56]:
udf=ev_df[usecolsX]

In [57]:
X=scalar.transform(udf)

In [58]:
# a function to create udf_y based on 3 classes: low, mid, high
def get_class(df,thre1,thre2):
    n=len(df)
    newdf=[]
    cut1=min([thre1,thre2])
    cut2=max([thre1,thre2])
    for item in df:
        if item<cut1: newdf+=[0]
        elif (item<cut2) and (item>=cut1):newdf+=[1]
        else: newdf+=[2]
    return newdf

In [59]:
y=get_class(ev_df['votes'].values,80,300)

In [60]:
y_pred=model.predict(X)

In [61]:
y_pred_prob=model.predict_proba(X)

In [62]:
result=list(zip(y,y_pred,ev_df['votes'].values,y_pred_prob[:,0],y_pred_prob[:,1],y_pred_prob[:,2]))

In [63]:
result_df=pd.DataFrame(result,columns=['class actual','class predicted','actual votes','probability in class0','probability in class1','probability in class2'])

In [64]:
result_df.sort_values(by=['actual votes'], ascending=False)

,class actual,class predicted,actual votes,probability in class0,probability in class1,probability in class2
0,2,0,5307,0.720179,0.037738,0.242083
1,2,0,487,0.654563,0.151468,0.193968
2,1,0,210,0.827639,0.067282,0.105079
3,1,2,199,0.153606,0.367613,0.478782
4,1,0,156,0.614444,0.311131,0.074425
9,1,0,128,0.738026,0.116429,0.145546
6,1,0,120,0.705840,0.216898,0.077262
7,1,1,117,0.276131,0.486468,0.237401
11,1,1,100,0.315582,0.523386,0.161032
5,1,1,96,0.155982,0.422431,0.421587


In [65]:
result_df.sort_values(by=['class predicted','probability in class0'], ascending=False)

,class actual,class predicted,actual votes,probability in class0,probability in class1,probability in class2
12,1,2,94,0.156706,0.166587,0.676706
3,1,2,199,0.153606,0.367613,0.478782
11,1,1,100,0.315582,0.523386,0.161032
10,1,1,95,0.282401,0.578611,0.138988
7,1,1,117,0.276131,0.486468,0.237401
16,0,1,53,0.263036,0.416766,0.320198
8,1,1,95,0.210288,0.483244,0.306468
5,1,1,96,0.155982,0.422431,0.421587
71,0,0,2,0.999789,0.000078,0.000133
98,0,0,0,0.999647,0.000181,0.000173


In [69]:
#  check the confusion matrix
a=precision_recall_fscore_support(y, y_pred)
print("confusion matrix:")
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))

confusion matrix:
[[86  1  0]
 [ 4  5  2]
 [ 2  0  0]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96        87
           1       0.83      0.45      0.59        11
           2       0.00      0.00      0.00         2

    accuracy                           0.91       100
   macro avg       0.59      0.48      0.52       100
weighted avg       0.90      0.91      0.90       100



In [67]:
f1s=a[2]
print("weighted average f1 score: ",(f1s[0]+f1s[1]*2+f1s[2]*3)/6)

weighted average f1 score:  0.3562274071639829


In [68]:
result_df.to_csv('102919_evdo512Pred_result.csv')